In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Dense, Dropout, Conv1D

from sklearn.metrics import f1_score, confusion_matrix

Using TensorFlow backend.


In [4]:
dataframe = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DM project/task1.train.txt', sep="\t", header=None, names=["text", "article_id", "label"])
dataframe.dropna()


,text,article_id,label
0,"Et tu, Rhody? A recent editorial in the Provi...",727600136,non-propaganda
1,A recent post in The Farmington Mirror — our t...,731714618,non-propaganda
2,"President Donald Trump, as he often does while...",731714635,non-propaganda
3,"February is Black History Month, and nothing l...",728627182,non-propaganda
4,"The snow was so heavy, whipped up by gusting w...",728627443,non-propaganda
...,...,...,...
35981,From The Telegraph: Towns in Brazil have beco...,785781276,propaganda
35982,The second episode of Consortium News on Flash...,785781322,propaganda
35983,"It is beginning. Actually, it’s been happenin...",785781455,propaganda
35984,Justin’s note: As regular Dispatch readers kno...,785781711,propaganda


In [0]:
dataframe['target'] = dataframe['label'].map({'propaganda': 1, 'non-propaganda': 0})

In [0]:
dataframe

,text,article_id,label,target
0,"Et tu, Rhody? A recent editorial in the Provi...",727600136,non-propaganda,0
1,A recent post in The Farmington Mirror — our t...,731714618,non-propaganda,0
2,"President Donald Trump, as he often does while...",731714635,non-propaganda,0
3,"February is Black History Month, and nothing l...",728627182,non-propaganda,0
4,"The snow was so heavy, whipped up by gusting w...",728627443,non-propaganda,0
...,...,...,...,...
35981,From The Telegraph: Towns in Brazil have beco...,785781276,propaganda,1
35982,The second episode of Consortium News on Flash...,785781322,propaganda,1
35983,"It is beginning. Actually, it’s been happenin...",785781455,propaganda,1
35984,Justin’s note: As regular Dispatch readers kno...,785781711,propaganda,1


In [6]:
ps = PorterStemmer()
nltk.download('stopwords')
cvec = CountVectorizer()
tokenizer = cvec.build_tokenizer()
def clean_text(str_input):
  str_input = str_input.replace("!"," exclamationsign")
  str_input = str_input.replace("?"," questionmark")
  tokens = tokenizer(str_input.lower())
  words = [ps.stem(token) for token in tokens if token not in stopwords.words('english')]
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
dataframe['article_prep'] = dataframe['text'].apply(clean_text)

In [0]:
df_dev, df_val = train_test_split(dataframe, test_size = 0.25, random_state = 42, stratify=dataframe['target'])


df_dev_x = df_dev['article_prep']
df_dev_y = df_dev['target'].ravel()


df_val_x = df_val['article_prep']
df_val_y = df_val['target'].ravel()


In [0]:
max_features = 6000
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(df_dev_x)

In [0]:


df_dev_x_tokens = tokenizer.texts_to_sequences(df_dev_x)
df_val_x_tokens = tokenizer.texts_to_sequences(df_val_x)



In [0]:
df_dev['article_prep'].apply(lambda x: len(x)).median()

2836.0

In [0]:
max_sequence_length=500

In [0]:
df_dev_x_pad = pad_sequences(df_dev_x_tokens, maxlen=max_sequence_length)
df_val_x_pad = pad_sequences(df_val_x_tokens, maxlen=max_sequence_length)

In [0]:
embed_size = 200

In [14]:
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Conv1D(100,3,padding='valid',strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         1200000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 100)         60100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2020      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 1,262,141


In [16]:
model.fit(df_dev_x_pad, df_dev_y, batch_size=128, epochs=2)




Epoch 1/2





26989/26989 [==============================] - 15s 538us/step - loss: 0.2609 - acc: 0.9132
Epoch 2/2
26989/26989 [==============================] - 7s 271us/step - loss: 0.0965 - acc: 0.9657


In [17]:
df_val_y_pred = model.predict(df_val_x_pad, verbose=1).round()


8997/8997 [==============================] - 1s 141us/step


In [18]:
print('F1-score: {0}'.format(f1_score(df_val_y_pred, df_val_y)))
confusion_matrix(df_val_y_pred, df_val_y)

F1-score: 0.8302277432712215


array([[7867,  203],
       [ 125,  802]])

In [19]:
score = metrics.accuracy_score(y_test, pred)
score_mlp_tfidf1=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)

NameError: ignored